In [16]:
from astroquery.jplhorizons import Horizons
from datetime import datetime, timedelta
import astropy.units as u
import re

def make_horizons_object(searchquery, observer='0', time='2023-11-03'):
    later_time = (datetime.strptime(time, '%Y-%m-%d') + timedelta(days=2)).strftime('%Y-%m-%d')
    object = Horizons(id=searchquery, location=observer, epochs={'start': time, 'stop':str(later_time),'step':'3d'}).vectors(get_raw_response=True)
    object_tab = Horizons(id=searchquery, location=observer, epochs={'start': time, 'stop':str(later_time),'step':'3d'}).vectors()
    print(object)
    m = re.search(r"\s+?Mass\s+?([\S]+).*?=\s+?([\S]+?)\s+?", object)
    r = re.search(r'\s+?radius\s+?([\S]+).*?=\s+?([\S]+?)\s+?', object)
    n = re.search(r"Target body name: (.+?) \((\d+)\)", object)
    if not m:  
        print(object)
        m = re.search(r"\s+?Mass,\s+?([\S]+).*?=\s+?([\S]+?)\s+?", object)
        mbase, mexp_s = m.groups()[1], m.groups()[0]
        print(mbase, mexp_s)
        mbase_n = ''.join(c for c in mbase if c.isdigit())
        mass = float(mbase_n) * 10**(float(mexp_s.split('10^')[1]))
    elif m:
        mbase, mexp_s = m.groups()[1], m.groups()[0]
        mass = float(mbase.split('+-')[0]) * 10**(float(mexp_s.split('x10^')[1]))
        print(mass)
    else: 
        print(object)
        raise LookupError('could not find Mass in output')
    if r:
        rad =r.groups()[1]
        radius = float(rad.split('+-')[0])*1000
        print(radius)
    else:
        raise LookupError('could not find radius (Vol. mean Radius) in output')
    if n:    
        name = f'{n.groups()[0]} ({n.groups()[1]})'
        print(name)
    else:
        print('could not find name for object, reverting to placeholder name.')
        name = None
    x, y, z = [object_tab[pos].quantity[0].to_value(u.m) for pos in ('x', 'y', 'z')]
    vx, vy, vz = [object_tab[pos].quantity[0].to_value(u.m/u.s) for pos in ('vx', 'vy', 'vz')]
    print(x,y,z, mass, name)
    #return Body(mass=mass, init_pos=(x,y,z), init_vel=(vx,vy,vz), radius=radius, identity=name)

make_horizons_object('Sun')
make_horizons_object('499')
make_horizons_object('999')
    
#print(bodies)

API VERSION: 1.2
API SOURCE: NASA/JPL Horizons API

*******************************************************************************
 Revised: July 31, 2013                  Sun                                 10

 PHYSICAL PROPERTIES (updated 2018-Aug-15):
  GM, km^3/s^2          = 132712440041.93938  Mass, 10^24 kg        = ~1988500
  Vol. mean radius, km  = 695700              Volume, 10^12 km^3    = 1412000
  Solar radius (IAU)    = 696000 km           Mean density, g/cm^3  = 1.408
  Radius (photosphere)  = 696500 km           Angular diam at 1 AU  = 1919.3"
  Photosphere temp., K  = 6600 (bottom)       Photosphere temp., K  = 4400(top)
  Photospheric depth    = ~500 km             Chromospheric depth   = ~2500 km
  Flatness, f           = 0.00005             Adopted sid. rot. per.= 25.38 d
  Surface gravity       =  274.0 m/s^2        Escape speed, km/s    =  617.7
  Pole (RA,DEC), deg.   = (286.13, 63.87)     Obliquity to ecliptic = 7.25 deg.
  Solar constant (1 AU) = 1367.6 W/m^2

ValueError: could not convert string to float: '~1988500'